In [1]:
#Mounting Google Drive from Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#Changing the current working directory to the Google Drive
#%cd /content/drive/My Drive/MLDL2024_project1-Enrico

In [3]:
#Importing the necessary libraries
import os
import torch
import numpy as np
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets.cityscapes import CityscapesCustom
from models.deeplabv2.deeplabv2 import get_deeplab_v2
from train import train_model
from utils import test_latency_FPS

In [4]:
#Set device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
#Create Dataloaders for Cityscapes
cityscapes_dir = os.path.dirname(os.getcwd()) + '/Cityscapes/Cityspaces/'

cityscapes_train_dataset = CityscapesCustom(cityscapes_dir, 'train')
cityscapes_test_dataset = CityscapesCustom(cityscapes_dir, 'val')

cityscapes_train_dataloader = DataLoader(cityscapes_train_dataset, batch_size=4, shuffle=True)
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=4, shuffle=False)

print(f'Train: {len(cityscapes_train_dataset)} images, divided into {len(cityscapes_train_dataloader)} batches of size {cityscapes_train_dataloader.batch_size}')
print(f'Test: {len(cityscapes_test_dataset)} images, divided into {len(cityscapes_test_dataloader)} batches of size {cityscapes_test_dataloader.batch_size}')

Train: 1572 images, divided into 393 batches of size 4
Test: 500 images, divided into 125 batches of size 4


In [6]:
#Testing Classic semantic segmentation network

#Set up the model with the pretrained weights
DeepLabV2_model = get_deeplab_v2().to(device)

#Set up the loss function and the optimizer
criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
optimizer = optim.Adam(DeepLabV2_model.parameters(), lr=1e-3)

#Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

Deeplab pretraining loading...


In [7]:
#Train the model
train_model(DeepLabV2_model, criterion, optimizer, cityscapes_train_dataloader, cityscapes_test_dataloader, device, n_epochs=10)

#Test latency and FPS
print(test_latency_FPS(DeepLabV2_model, 512, 1024, device))

Epoch 1/10, IoU: 60.27%, Intersection: 22922, Union: 38031
Epoch 2/10, IoU: 54.74%, Intersection: 20818, Union: 38031
Epoch 3/10, IoU: 53.92%, Intersection: 20508, Union: 38031
Epoch 4/10, IoU: 51.91%, Intersection: 19742, Union: 38031
Epoch 5/10, IoU: 60.06%, Intersection: 22842, Union: 38031
Epoch 6/10, IoU: 58.81%, Intersection: 22367, Union: 38031
Epoch 7/10, IoU: 67.35%, Intersection: 25612, Union: 38031
Epoch 8/10, IoU: 67.22%, Intersection: 25563, Union: 38031
Epoch 9/10, IoU: 73.96%, Intersection: 28126, Union: 38031
Epoch 10/10, IoU: 75.31%, Intersection: 28642, Union: 38031
